In [281]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from bokeh.charts import Scatter, BoxPlot, save, output_file, show
from bokeh.embed import autoload_static
from bokeh.resources import CDN
from bokeh.palettes import Accent3
from bokeh.io import output_notebook, output_file

import os
import pickle

# Cell Surface Proteins

Look for overexpressed cell surface proteins and compare to known drug targets

- For each tissue group
    - Split into two groups to compare
        - Compute log2 FC for every gene between the two groups
        - Plot barplots of all genes with positive L2FC
        - ????
        - Profit

In [282]:
df = pd.read_csv('/mnt/rna-seq-analysis/data/xena/deseq2_normalized_tcga_gtex_counts.tsv', sep='\t', index_col=0)
cspa = pd.read_csv('/mnt/rna-seq-analysis/metadata/CSPA_high_confidence.tsv', sep='\t', index_col=0)

In [283]:
df.head()

,TCGA-AD-5900-01,TCGA-BP-4968-01,TCGA-NG-A4VU-01,TCGA-CG-4305-01,TCGA-AO-A03M-01,TCGA-ZH-A8Y6-01,TCGA-HT-7686-01,TCGA-BR-6458-11,TCGA-29-1699-01,TCGA-KK-A6E1-01,...,GTEX-ZUA1-0226-SM-5NQ9Q,GTEX-Q734-0526-SM-2I3EH,GTEX-Y5V6-0526-SM-4VBRV,GTEX-1192X-3126-SM-5N9BY,GTEX-13X6H-0526-SM-5LU4Q,K-562-SM-3MJHH,GTEX-11GSO-2326-SM-5A5LX,GTEX-YJ8A-1126-SM-5IFJU,GTEX-RU1J-0226-SM-2TF5Y,GTEX-12BJ1-0426-SM-5FQSO
ENSG00000116032.5,4.381288,1.511815,1.430657,0.986667,1.945771,0.903716,0.000000,0.000000,8.441424,3.926956,...,14.301939,77.979912,15.608325,8.313450,31.846108,15.292863,14.530338,17.417563,17.747865,17.676720
ENSG00000171174.13,87.625753,656.631497,133.766402,154.413440,128.420862,232.255080,128.729659,154.710834,341.643175,244.453014,...,150.170357,172.455574,272.625414,137.765738,130.569042,16.822149,61.350314,592.197153,133.108991,229.797358
ENSG00000149136.7,7932.321326,4682.089977,7258.436790,3999.949436,5836.533582,2959.670762,8468.201070,3971.274572,6338.336754,8266.242476,...,5958.545231,6014.950501,4380.736621,3552.218287,6362.852322,8834.176959,4249.316508,6886.904540,5096.595360,6787.860414
ENSG00000078237.5,196.062623,330.583471,273.255431,533.787036,200.025221,212.373322,576.032718,278.915306,336.719011,295.503442,...,294.977487,245.936645,339.220935,263.655119,128.976736,29.566201,416.536345,404.087469,221.848318,203.282278
ENSG00000146083.11,1343.412332,2331.722131,2469.313472,1886.014639,1978.459577,2124.185077,1780.760286,2426.345260,5307.076163,2677.202284,...,3019.496818,3705.545422,2409.925419,2769.566381,3585.871729,2052.811922,2504.061513,4756.736545,3531.825224,3517.667246


In [310]:
cspa.head()

,Symbol,Description
ENSG00000139304,PTPRQ,"protein tyrosine phosphatase, receptor type Q(..."
ENSG00000084674,APOB,apolipoprotein B(APOB)
ENSG00000121594,CD80,CD80 molecule(CD80)
ENSG00000114013,CD86,CD86 molecule(CD86)
ENSG00000178562,CD28,CD28 molecule(CD28)


### Subset for CSPA genes

Map gene IDS to gene names

In [222]:
gene_map = pickle.load(open('/mnt/rna-seq-analysis/data/objects/gene_map.pickle', 'rb'))

In [223]:
genes = [gene_map[x] if x in gene_map else x for x in df.index]
df.index = genes

In [224]:
cspa_genes = [x for x in cspa.Symbol if x in df.index]
len(cspa_genes)

964

In [311]:
'HER2' in cspa_genes

False

In [225]:
df = df.T  # Transpose to features by samples
df.head(1)

,GRIN3B,RBKS,SSRP1,TIGAR,RNF44,UQCC2,ADGRD2,C11orf84,SPAG6,RPL23A,...,CREB3L3,HEATR6,BATF3,GGNBP2,CHI3L2,HMBOX1,IFNK,PPP6R1,C19orf48,OR8D4
TCGA-AD-5900-01,4.381288,87.625753,7932.321326,196.062623,1343.412332,712.506907,0.0,337.359151,0.0,17819.792274,...,2.738305,236.589534,36.693284,1129.824558,69.005281,145.130154,0.0,2655.607989,2991.324156,0.0


In [38]:
df_cspa = df[cspa_genes]

In [231]:
def get_tissues():
    pair_file = '/home/ubuntu/rnaseq-recompute-analysis/preprocessing/candidate_tissues.csv'
    tissues = []
    with open(pair_file, 'r') as f:
        for line in f:
            line = line.strip().split(',') if ',' in line else [line.strip()]
            tissues.append(line)
    return tissues

def flatten(x):
    """
    Flattens a nested array into a single list

    :param list x: The nested list/tuple to be flattened.
    """
    result = []
    for el in x:
        if hasattr(el, "__iter__") and not isinstance(el, basestring):
            result.extend(flatten(el))
        else:
            result.append(el)
    return result

In [232]:
tissues = get_tissues()

In [233]:
metadata = pd.read_csv('/mnt/rna-seq-analysis/metadata/tcga_gtex_metadata_intersect.tsv', index_col=0, sep='\t')

In [240]:
def get_l2fc(a, b):
    return np.log2(b + 1) - np.log2(a + 1)

In [248]:
def make_plots(sub, subdir, d1, d2, n1, n2, name):
    mean = sub.mean(axis=0)
    # print '\n{}:\t{}\n{}:\t{}'.format(n1, len(d1), n2, len(d2))
    d1_medians = sub.loc[d1].median(axis=0)
    d2_medians = sub.loc[d2].median(axis=0)
    l2fc = get_l2fc(d1_medians, d2_medians)
    plot = pd.DataFrame()
    plot['mean'] = np.log2(mean + 1)
    plot['l2fc'] = l2fc
    plot['gene'] = sub.columns
    plot.index = sub.columns
    bot25_genes = list(plot.sort_values('l2fc').index)[:25]
    
    colorby = []
    for i, gene in enumerate(sub.columns):
        if gene in cspa_genes:
            colorby.append('CSPA')
        elif np.abs(l2fc[i]) > 2:
            colorby.append('sig')
        else:
            colorby.append('unsig')
    
    plot['colorby'] = colorby

    tooltips = [
        ('Gene', '@gene'),
        ('Mean', '@mean'),
        ('L2FC', '@l2fc')]
    p = Scatter(plot, x='mean', y='l2fc', title=name,
               xlabel='Log2(Mean)', ylabel='Log2(FC)', 
               color='colorby', 
               tooltips=tooltips,
               legend=True,
               palette=Accent3, 
               responsive=True)
    p.title.align = 'center'
    output_file = os.path.join(subdir, 'MA', name + '.html')
    save(p, output_file)
    
    # Boxplots of top overexpressed (B respect to A)
    top_genes = list(plot.sort_values('l2fc', ascending=False).index)
    top_genes = [x for x in top_genes if x in cspa_genes][:25]
    top1 = sub.loc[d1][top_genes]
    top2 = sub.loc[d2][top_genes]

    plot = pd.DataFrame()
    plot['Log2(Counts)'] = map(lambda x: np.log2(float(x) + 1), 
                         flatten([top1.loc[x] for x in top1.index] + [top2.loc[x] for x in top2.index]))
    plot['gene'] = list(top1.columns) * len(top1.index) + list(top2.columns) * len(top2.index)
    plot['dataset'] = [n1 for _ in xrange(top1.shape[0] * top1.shape[1])] + \
                      [n2 for _ in xrange(top2.shape[0] * top2.shape[1])]

    tooltips = [('# Samples', '@height')]
    b = BoxPlot(plot, label='gene', values='Log2(Counts)', group='dataset',
               tooltips=tooltips,
               responsive=True,
               title=name,
               color='dataset',)
    b.title.align = 'center'
    output_file = os.path.join(subdir, 'Box', name + '.html')
    save(b, output_file)

In [247]:
dirs = ['G_T', 'N_T', 'G+N_T', 'G_N', 'G_T+N']
for d in dirs:
    if not os.path.isdir(d):
        os.mkdir(d)
        os.mkdir(os.path.join(d, 'MA'))
        os.mkdir(os.path.join(d, 'Box'))

for tissue in tissues:
    name = '-'.join(tissue)
    samples = flatten([list(set(metadata[metadata.tissue == x].index) for x in tissue)])
    samples = [x for x in samples if x in df_cspa.index]
    # sub = df_cspa.loc[samples]
    sub = df.loc[samples]
    print 'Tissue: {}\tSamples: {}\tGenes: {}'.format(name, sub.shape[0], sub.shape[1])
    g = list(set([x for x in samples if not x.startswith('TCGA')]))
    t = list(set([x for x in samples if x.startswith('TCGA') and x.endswith('01')]))
    n = list(set([x for x in samples if x.startswith('TCGA') and x.endswith('11')]))
    if g and t:
        make_plots(sub, 'G_T', g, t, 'GTEx', 'Tumor', name)
    if n and t:
        make_plots(sub, 'N_T', n, t, 'Normal', 'Tumor', name)
    if g and n and t:
        make_plots(sub, 'G+N_T', g+n, t, 'GTEx+Normal', 'Tumor', name)
        make_plots(sub, 'G_T+N', g, t+n, 'GTEx', 'Tumor+Normal', name)
    if g and n:
        make_plots(sub, 'G_N', g, n, 'GTEx', 'Normal', name)

Tissue: Adrenal	Samples: 387	Genes: 19797
Tissue: Bladder	Samples: 435	Genes: 19797
Tissue: Brain	Samples: 1836	Genes: 19797
Tissue: Breast	Samples: 1387	Genes: 19797
Tissue: Cervix	Samples: 318	Genes: 19797
Tissue: Colon-Small_intestine	Samples: 828	Genes: 19797
Tissue: Esophagus	Samples: 845	Genes: 19797
Tissue: Kidney	Samples: 1040	Genes: 19797
Tissue: Liver	Samples: 531	Genes: 19797
Tissue: Lung	Samples: 1409	Genes: 19797
Tissue: Ovary	Samples: 515	Genes: 19797
Tissue: Pancreas	Samples: 348	Genes: 19797
Tissue: Prostate	Samples: 646	Genes: 19797
Tissue: Skin-Head	Samples: 1572	Genes: 19797
Tissue: Stomach	Samples: 622	Genes: 19797
Tissue: Testis	Samples: 319	Genes: 19797
Tissue: Thyroid	Samples: 849	Genes: 19797
Tissue: Uterus	Samples: 339	Genes: 19797


Across Datasets

In [216]:
samples = []
for tissue in tissues:
    name = '-'.join(tissue)
    temp = flatten([list(set(metadata[metadata.tissue == x].index) for x in tissue)])
    samples.extend([x for x in temp if x in df_cspa.index])
sub = df_cspa.loc[samples]
print 'Samples: {}\tGenes: {}'.format(sub.shape[0], sub.shape[1])
g = list(set([x for x in samples if not x.startswith('TCGA')]))
t = list(set([x for x in samples if x.startswith('TCGA') and x.endswith('01')]))
n = list(set([x for x in samples if x.startswith('TCGA') and x.endswith('11')]))

Samples: 14226	Genes: 968


In [217]:
make_plots(sub, 'G_T', g, t, 'GTEx', 'Tumor', 'All')
make_plots(sub, 'N_T', n, t, 'Normal', 'Tumor', 'All')
make_plots(sub, 'G+N_T', g+n, t, 'GTEx+Normal', 'Tumor', 'All')
make_plots(sub, 'G_T+N', g, t+n, 'GTEx', 'Tumor+Normal', 'All')
make_plots(sub, 'G_N', g, n, 'GTEx', 'Normal', 'All')

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log2
  


# Generate Negative Control MA Plots
